This is a file to clean the CSV files scraped from the NOAA API

In [ ]:
import pandas as pd 
import numpy as np 
from MI_info import stationids_to_FIPS

In [ ]:
# The raw data we want to clean
file_name = 'file_name_goes_here'

In [ ]:
df = pd.read_csv(file_name)
df = df.set_index('station')
df.columns.name = 'date'

In [ ]:
duplicates_exist = False
for i in df.index.duplicated():
    if i: 
        duplicates_exist = True 
print(f'duplicates_exist: {duplicates_exist}')

Now we create a county-level rainfall CSV

In [ ]:
for station in df.index:
    try: 
        prefix, station = station.split(':')
        fips = stationids_to_FIPS[station]
    except KeyError: 
        fips = np.nan
    df.loc[prefix + ':' + station, 'FIPS'] = fips

In [ ]:
# Rearrange columns so that FIPS is first 
col = df.columns
col = col[:-1]
col = col.insert(0, 'FIPS')
df = df[col]
df.head()

In [ ]:
# How many stations do not have an assigned FIPS? 
num_unassigned = len([x for x in df.FIPS.notna() if x == False])
print(f'There are {num_unassigned} stations with no FIPS')

In [ ]:
df_fips = df[df['FIPS'].notna()].set_index('FIPS')

In [ ]:
# Docs say that mean() skips NaN values by default
df_fips_mean = df_fips.groupby(['FIPS']).mean()

In [ ]:
# Give column headers the appropriate datetime format
df_fips_mean.columns = pd.to_datetime(df_fips_mean.columns)
df_fips_mean.head()

In [ ]:
# Uncomment to save as CSV
fips_file_name_1 = file_name.split('.')[0] + '_FIPS.csv'
df_fips_mean.to_csv(fips_file_name_1)

Now we need to mess around with the DataFrame until it becomes an easy-to-use format for Tableau

In [ ]:
df_reformatted = pd.DataFrame(df_fips_mean.unstack())

In [ ]:
new_df = pd.DataFrame()

In [ ]:
i = 0
for code in df_fips_mean.index:
    print("Working on FIPS code:", code)
    for date in df_fips_mean.columns:
        new_df.loc[i, 'FIPS'] = code
        new_df.loc[i, 'date'] = date 
        date_str = str(date).split(' ')[0]
        new_df.loc[i, 'rainfall'] = df_fips_mean.loc[code, date_str]
        i += 1

In [ ]:
new_df['FIPS'] = new_df['FIPS'].apply(np.int64).apply(str)
new_df.index.name = 'row_no'

In [ ]:
new_df.head()

In [ ]:
# # Uncomment to save as CSV
# fips_file_name_2 = file_name.split('.')[0] + '_alt_formatting.csv'
# new_df.to_csv(fips_file_name_2)